In [1]:
import sys
sys.path.append("../")


In [6]:
from sqlmodel import Session, select
from models.vault_performance import VaultPerformance
import pandas as pd
from core.db import engine

session = Session(engine)

query = """
SELECT datetime,total_locked_value
FROM public.vault_performance
WHERE vault_id = '65f75bd7-a2d2-4764-ae31-78e4bb132c62'
ORDER BY datetime ASC
"""

# Execute the query and load the results into a DataFrame
df = pd.read_sql(query, session.bind)
df["datetime"] = pd.to_datetime(df["datetime"])

# Set the datetime column as the index
df.set_index("datetime", inplace=True)

# Resample to daily frequency, keeping the first value for each day
daily_df = df.resample("D").first()

# Forward-fill missing values
daily_df.ffill(inplace=True)

# Reset the index if needed
daily_df.reset_index(inplace=True)

daily_df["datetime"] = daily_df["datetime"].dt.strftime("%Y-%m-%d")
# Display the resulting DataFrame
daily_df

,datetime,total_locked_value
0,2024-06-04,150.115912
1,2024-06-05,150.083810
2,2024-06-06,150.070816
3,2024-06-07,716.565137
4,2024-06-08,717.030704
...,...,...
162,2024-11-13,75833.344024
163,2024-11-14,76969.476964
164,2024-11-15,77129.887568
165,2024-11-16,77080.018347


In [8]:
ALLOCATION_RATIO: float = 1 / 2
AE_USD = 0.08/365
LST_YEILD = 0.036/365

In [ ]:
import requests


def get_funding_history(
    instrument_name: str = "ETH-PERP",
    start_time: int = 0,
    end_time: int = None,
    limit: int = 10,
):
    url = "https://api.aevo.xyz/funding-history"
    params = {
        "instrument_name": instrument_name,
        "start_time": start_time,
        "end_time": end_time,
        "limit": limit,
    }
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if "funding_history" in data and len(data["funding_history"]) > 0:
            first_item = data["funding_history"][0]
            funding_rate = float(first_item[2])  
            return funding_rate
        else:
            return 0 
    else:
        return 0

In [16]:
from datetime import datetime, timedelta


def convert_to_nanoseconds(datetime: datetime) -> int:
    """
    Converts a date string in the format 'YYYY-MM-DD' to a UNIX timestamp in nanoseconds.

    Args:
        date_string (str): The date string to convert (e.g., '2024-06-05').

    Returns:
        int: The UNIX timestamp in nanoseconds.
    """
    nanoseconds = int(datetime.timestamp() * 1e9)
    return nanoseconds

In [40]:
from datetime import datetime, timedelta

funding_rate_dt = []
# foreach row in df1, calculate the roi
for i, row in daily_df.iterrows():
    dt =  datetime.strptime(daily_df.iloc[i]["datetime"], "%Y-%m-%d")
    end_date = convert_to_nanoseconds(dt)
    start_dt = dt - timedelta(days=1)
    start_date = convert_to_nanoseconds(start_dt)
    funding_history = get_funding_history(start_time=start_date, end_time=end_date)
    if i == 0:
        funding_rate_dt.append(
            {
                "datetime": dt,
                "total_locked_value": row["total_locked_value"],
                "funding_rate": funding_history,
                "LST_YEILD": 0,
                "AE_USD": 0,
                "funding": 0,
            }
        )

        continue

    total_locked_value= float(daily_df.iloc[i-1]["total_locked_value"])

    funding_rate_dt.append(
        {
            "datetime": dt,
            "total_locked_value": row["total_locked_value"],
            "funding_rate": funding_history,
            "LST_YEILD": LST_YEILD * ALLOCATION_RATIO * total_locked_value,
            "AE_USD": AE_USD * ALLOCATION_RATIO * total_locked_value,
            "funding": funding_history * ALLOCATION_RATIO * 24 * total_locked_value,
        }
    )

funding_rate_df = pd.DataFrame(funding_rate_dt)

# In ra DataFrame
print(funding_rate_df)

      datetime  total_locked_value  funding_rate  LST_YEILD    AE_USD  \
0   2024-06-04          150.115912     -0.000006   0.000000  0.000000   
1   2024-06-05          150.083810      0.000012   0.007403  0.016451   
2   2024-06-06          150.070816      0.000043   0.007401  0.016448   
3   2024-06-07          716.565137      0.000043   0.007401  0.016446   
4   2024-06-08          717.030704      0.000036   0.035337  0.078528   
..         ...                 ...           ...        ...       ...   
162 2024-11-13        75833.344024      0.000013   2.519941  5.599870   
163 2024-11-14        76969.476964      0.000013   3.739727  8.310503   
164 2024-11-15        77129.887568      0.000013   3.795755  8.435011   
165 2024-11-16        77080.018347      0.000013   3.803666  8.452590   
166 2024-11-17        87114.766978      0.000013   3.801206  8.447125   

       funding  
0     0.000000  
1     0.021617  
2     0.077443  
3     0.077437  
4     0.309556  
..         ...  
162 

In [42]:
funding_rate_df.to_csv("D:\\funding_rate_data.csv", index=False)